In [ ]:
# ! git clone https://github.com/google-research/maxim/
# %cd ./maxim

# !pip install -r requirements.txt
# !pip install --upgrade jax
# !pip install gdown
# !pip uninstall jax

# !python setup.py build
# !python setup.py install

In [ ]:
# import sys   
# !{sys.executable} -m pip install pillow matplotlib cog

In [1]:
cd maxim

/home/j-k8e104/maxim


In [2]:
from PIL import Image

import matplotlib.pyplot as plt
import collections
import importlib
import io
import os
import math
import requests
from tqdm import tqdm
import gdown # to download weights from Drive

import flax
import jax.numpy as jnp
import ml_collections
import numpy as np
import tensorflow as tf
from jax.experimental import jax2tf

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

# below code lines are from run_eval.py
_MODEL_FILENAME = 'maxim'

_MODEL_VARIANT_DICT = {
    'Denoising': 'S-3',
    'Deblurring': 'S-3',
    'Deraining': 'S-2',
    'Dehazing': 'S-2',
    'Enhancement': 'S-2',
}

_MODEL_CONFIGS = {
    'variant': '',
    'dropout_rate': 0.0,
    'num_outputs': 3,
    'use_bias': True,
    'num_supervision_scales': 3,
}


In [3]:
def sizeof_fmt(size, suffix='B'):
    """Get human readable file size.
    Args:
        size (int): File size.
        suffix (str): Suffix. Default: 'B'.
    Return:
        str: Formated file siz.
    """
    for unit in ['', 'K', 'M', 'G', 'T', 'P', 'E', 'Z']:
        if abs(size) < 1024.0:
            return f'{size:3.1f} {unit}{suffix}'
        size /= 1024.0
    return f'{size:3.1f} Y{suffix}'


def download_file_from_google_drive(file_id, save_path):
    """Download files from google drive.

    Ref:
    https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drive  # noqa E501

    Args:
        file_id (str): File id.
        save_path (str): Save path.
    """

    session = requests.Session()
    URL = 'https://docs.google.com/uc?export=download'
    params = {'id': file_id}

    response = session.get(URL, params=params, stream=True)
    token = get_confirm_token(response)
    if token:
        params['confirm'] = token
        response = session.get(URL, params=params, stream=True)

    # get file size
    response_file_size = session.get(
        URL, params=params, stream=True, headers={'Range': 'bytes=0-2'})
    if 'Content-Range' in response_file_size.headers:
        file_size = int(
            response_file_size.headers['Content-Range'].split('/')[1])
    else:
        file_size = None

    save_response_content(response, save_path, file_size)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None


def save_response_content(response,
                          destination,
                          file_size=None,
                          chunk_size=32768):
    if file_size is not None:
        pbar = tqdm(total=math.ceil(file_size / chunk_size), unit='chunk')

        readable_file_size = sizeof_fmt(file_size)
    else:
        pbar = None

    with open(destination, 'wb') as f:
        downloaded_size = 0
        for chunk in response.iter_content(chunk_size):
            downloaded_size += chunk_size
            if pbar is not None:
                pbar.update(1)
                pbar.set_description(f'Download {sizeof_fmt(downloaded_size)} '
                                     f'/ {readable_file_size}')
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)
        if pbar is not None:
            pbar.close()

In [4]:

def resize(path, new_width_height = 1280, save_image = False, convert_RGB = True, clip_full_hd = False, quality = 100):
  '''
  Resize and return Given Image
  args:
    path: Image Path
    new_width_height = Reshaped image's width and height. # If integer is given, it'll keep the aspect ratio as it is by shrinking the Bigger dimension (width or height) to the max of new_width_height  and then shring the smaller dimension accordingly 
    save_image = Whether to save the image or not
    convert_RGB: Whether to Convert the RGBA image to RGB (by default backgroud is white)
  '''
  image = Image.open(path)
  w, h = image.size

  fixed_size = new_width_height if isinstance(new_width_height, int) else False

  if fixed_size:
    if h > w:
      fixed_height = fixed_size
      height_percent = (fixed_height / float(h))
      width_size = int((float(w) * float(height_percent)))
      image = image.resize((width_size, fixed_height), Image.NEAREST)

    else:
      fixed_width = fixed_size
      width_percent = (fixed_width / float(w))
      height_size = int((float(h) * float(width_percent)))
      image = image.resize((fixed_width, height_size), Image.NEAREST) # Try Image.ANTIALIAS inplace of Image.NEAREST

  else:
    image = image.resize(new_width_height)

  if image.mode == "RGBA" and convert_RGB:
    # image.load() # required for png.split()
    # new = Image.new("RGB", image.size, (255, 255, 255)) # White Background
    # image = new.paste(image, mask=image.split()[3]) # 3 is the alpha channel

    new = Image.new("RGBA", image.size, "WHITE") # Create a white rgba background
    new.paste(image, (0, 0), image) # Paste the image on the background.
    image = new.convert('RGB')

  if save_image:
    image.save(path, quality = quality)

  return image


class DummyFlags():
  def __init__(self, ckpt_path:str, task:str, input_dir: str = "./maxim/images/Enhancement", output_dir:str = "./maxim/images/Results", has_target:bool = False, save_images:bool = True, geometric_ensemble:bool = False):
    '''
    Builds the dummy flags which replicates the behaviour of Terminal CLI execution (same as ArgParse)
    args:
      ckpt_path: Saved Model CheckPoint: Find all the checkpoints for pre trained models at https://console.cloud.google.com/storage/browser/gresearch/maxim/ckpt/
      task: Task for which the model waas trained. Each task uses different Data and Checkpoints. Find the details of tasks and respective checkpoints details at: https://github.com/google-research/maxim#results-and-pre-trained-models
      input_dir: Input Directory. We do not need it here as we are directly passing one image at a time
      output_dir: Also not needed in out code
      has_target: Used to calculate PSNR and SSIM calculation. Not needed in our case
      save_images: Used in CLI command where images were saved in loop. Not needed in our case
      geometric_ensemble: Was used in training part and as it is just an Inference part, it is not needed

    '''
    self.ckpt_path = ckpt_path
    self.task = task
    self.input_dir = input_dir
    self.output_dir = output_dir
    self.has_target = has_target
    self.save_images = save_images
    self.geometric_ensemble = geometric_ensemble


In [5]:
# Copyright 2022 Google LLC.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


def recover_tree(keys, values):
  """Recovers a tree as a nested dict from flat names and values.

  This function is useful to analyze checkpoints that are saved by our programs
  without need to access the exact source code of the experiment. In particular,
  it can be used to extract an reuse various subtrees of the scheckpoint, e.g.
  subtree of parameters.
  Args:
    keys: a list of keys, where '/' is used as separator between nodes.
    values: a list of leaf values.
  Returns:
    A nested tree-like dict.
  """
  tree = {}
  sub_trees = collections.defaultdict(list)
  for k, v in zip(keys, values):
    if '/' not in k:
      tree[k] = v
    else:
      k_left, k_right = k.split('/', 1)
      sub_trees[k_left].append((k_right, v))
  for k, kv_pairs in sub_trees.items():
    k_subtree, v_subtree = zip(*kv_pairs)
    tree[k] = recover_tree(k_subtree, v_subtree)
  return tree


def mod_padding_symmetric(image, factor=64):
  """Padding the image to be divided by factor."""
  height, width = image.shape[0], image.shape[1]
  height_pad, width_pad = ((height + factor) // factor) * factor, (
      (width + factor) // factor) * factor
  padh = height_pad - height if height % factor != 0 else 0
  padw = width_pad - width if width % factor != 0 else 0
  image = jnp.pad(
      image, [(padh // 2, padh // 2), (padw // 2, padw // 2), (0, 0)],
      mode='reflect')
  return image


def get_params(ckpt_path):
  """Get params checkpoint."""

  with tf.io.gfile.GFile(ckpt_path, 'rb') as f:
    data = f.read()
  values = np.load(io.BytesIO(data))
  params = recover_tree(*zip(*values.items()))
  params = params['opt']['target']

  return params


def calculate_psnr(img1, img2, crop_border, test_y_channel=False):
  """Calculate PSNR (Peak Signal-to-Noise Ratio).

  Ref: https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio
  Args:
    img1 (ndarray): Images with range [0, 255].
    img2 (ndarray): Images with range [0, 255].
    crop_border (int): Cropped pixels in each edge of an image. These
        pixels are not involved in the PSNR calculation.
    test_y_channel (bool): Test on Y channel of YCbCr. Default: False.
  Returns:
    float: psnr result.
  """
  assert img1.shape == img2.shape, (
      f'Image shapes are differnet: {img1.shape}, {img2.shape}.')
  img1 = img1.astype(np.float64)
  img2 = img2.astype(np.float64)

  if crop_border != 0:
    img1 = img1[crop_border:-crop_border, crop_border:-crop_border, ...]
    img2 = img2[crop_border:-crop_border, crop_border:-crop_border, ...]

  if test_y_channel:
    img1 = to_y_channel(img1)
    img2 = to_y_channel(img2)

  mse = np.mean((img1 - img2)**2)
  if mse == 0:
    return float('inf')
  return 20. * np.log10(255. / np.sqrt(mse))


def _convert_input_type_range(img):
  """Convert the type and range of the input image.

  It converts the input image to np.float32 type and range of [0, 1].
  It is mainly used for pre-processing the input image in colorspace
  convertion functions such as rgb2ycbcr and ycbcr2rgb.
  Args:
    img (ndarray): The input image. It accepts:
        1. np.uint8 type with range [0, 255];
        2. np.float32 type with range [0, 1].
  Returns:
      (ndarray): The converted image with type of np.float32 and range of
          [0, 1].
  """
  img_type = img.dtype
  img = img.astype(np.float32)
  if img_type == np.float32:
    pass
  elif img_type == np.uint8:
    img /= 255.
  else:
    raise TypeError('The img type should be np.float32 or np.uint8, '
                    f'but got {img_type}')
  return img


def _convert_output_type_range(img, dst_type):
  """Convert the type and range of the image according to dst_type.

  It converts the image to desired type and range. If `dst_type` is np.uint8,
  images will be converted to np.uint8 type with range [0, 255]. If
  `dst_type` is np.float32, it converts the image to np.float32 type with
  range [0, 1].
  It is mainly used for post-processing images in colorspace convertion
  functions such as rgb2ycbcr and ycbcr2rgb.
  Args:
    img (ndarray): The image to be converted with np.float32 type and
        range [0, 255].
    dst_type (np.uint8 | np.float32): If dst_type is np.uint8, it
        converts the image to np.uint8 type with range [0, 255]. If
        dst_type is np.float32, it converts the image to np.float32 type
        with range [0, 1].
  Returns:
    (ndarray): The converted image with desired type and range.
  """
  if dst_type not in (np.uint8, np.float32):
    raise TypeError('The dst_type should be np.float32 or np.uint8, '
                    f'but got {dst_type}')
  if dst_type == np.uint8:
    img = img.round()
  else:
    img /= 255.

  return img.astype(dst_type)


def rgb2ycbcr(img, y_only=False):
  """Convert a RGB image to YCbCr image.

  This function produces the same results as Matlab's `rgb2ycbcr` function.
  It implements the ITU-R BT.601 conversion for standard-definition
  television. See more details in
  https://en.wikipedia.org/wiki/YCbCr#ITU-R_BT.601_conversion.
  It differs from a similar function in cv2.cvtColor: `RGB <-> YCrCb`.
  In OpenCV, it implements a JPEG conversion. See more details in
  https://en.wikipedia.org/wiki/YCbCr#JPEG_conversion.

  Args:
    img (ndarray): The input image. It accepts:
        1. np.uint8 type with range [0, 255];
        2. np.float32 type with range [0, 1].
    y_only (bool): Whether to only return Y channel. Default: False.
  Returns:
    ndarray: The converted YCbCr image. The output image has the same type
        and range as input image.
  """
  img_type = img.dtype
  img = _convert_input_type_range(img)
  if y_only:
    out_img = np.dot(img, [65.481, 128.553, 24.966]) + 16.0
  else:
    out_img = np.matmul(img,
                        [[65.481, -37.797, 112.0], [128.553, -74.203, -93.786],
                         [24.966, 112.0, -18.214]]) + [16, 128, 128]
  out_img = _convert_output_type_range(out_img, img_type)
  return out_img


def to_y_channel(img):
  """Change to Y channel of YCbCr.

  Args:
    img (ndarray): Images with range [0, 255].
  Returns:
    (ndarray): Images with range [0, 255] (float type) without round.
  """
  img = img.astype(np.float32) / 255.
  if img.ndim == 3 and img.shape[2] == 3:
    img = rgb2ycbcr(img, y_only=True)
    img = img[..., None]
  return img * 255.


def augment_image(image, times=8):
  """Geometric augmentation."""
  if times == 4:  # only rotate image
    images = []
    for k in range(0, 4):
      images.append(np.rot90(image, k=k))
    images = np.stack(images, axis=0)
  elif times == 8:  # roate and flip image
    images = []
    for k in range(0, 4):
      images.append(np.rot90(image, k=k))
    image = np.fliplr(image)
    for k in range(0, 4):
      images.append(np.rot90(image, k=k))
    images = np.stack(images, axis=0)
  else:
    raise Exception(f'Error times: {times}')
  return images


def deaugment_image(images, times=8):
  """Reverse the geometric augmentation."""

  if times == 4:  # only rotate image
    image = []
    for k in range(0, 4):
      image.append(np.rot90(images[k], k=4-k))
    image = np.stack(image, axis=0)
    image = np.mean(image, axis=0)
  elif times == 8:  # roate and flip image
    image = []
    for k in range(0, 4):
      image.append(np.rot90(images[k], k=4-k))
    for k in range(0, 4):
      image.append(np.fliplr(np.rot90(images[4+k], k=4-k)))
    image = np.mean(image, axis=0)
  else:
    raise Exception(f'Error times: {times}')
  return image


def is_image_file(filename):
  """Check if it is an valid image file by extension."""
  return any(
      filename.endswith(extension)
      for extension in ['jpeg', 'JPEG', 'jpg', 'png', 'JPG', 'PNG', 'gif'])


def save_img(img, pth):
  """Save an image to disk.

  Args:
    img: jnp.ndarry, [height, width, channels], img will be clipped to [0, 1]
      before saved to pth.
    pth: string, path to save the image to.
  """
  Image.fromarray(np.array(
      (np.clip(img, 0., 1.) * 255.).astype(jnp.uint8))).save(pth, 'PNG')


def make_shape_even(image):
  """Pad the image to have even shapes."""
  height, width = image.shape[0], image.shape[1]
  padh = 1 if height % 2 != 0 else 0
  padw = 1 if width % 2 != 0 else 0
  image = jnp.pad(image, [(0, padh), (0, padw), (0, 0)], mode='reflect')
  return image


# Refactored code --------------------------------------------------------------------------------------------------------------------

def build_model(task = "Enhancement"):
  model_mod = importlib.import_module(f'maxim.models.{_MODEL_FILENAME}')
  model_configs = ml_collections.ConfigDict(_MODEL_CONFIGS)

  model_configs.variant = _MODEL_VARIANT_DICT[task]

  model = model_mod.Model(**model_configs)
  return model


def pre_process(input_file):
  '''
  Pre-process the image before sending to the model
  '''
  input_img = np.asarray(Image.open(input_file).convert('RGB'),np.float32) / 255.
  # Padding images to have even shapes
  height, width = input_img.shape[0], input_img.shape[1]
  input_img = make_shape_even(input_img)
  height_even, width_even = input_img.shape[0], input_img.shape[1]

  # padding images to be multiplies of 64
  input_img = mod_padding_symmetric(input_img, factor=64)
  input_img = np.expand_dims(input_img, axis=0)

  return input_img, height, width, height_even, width_even


def predict(input_img):
  # handle multi-stage outputs, obtain the last scale output of last stage
  return model.apply({'params': flax.core.freeze(params)}, input_img)


def post_process(preds, height, width, height_even, width_even):
  '''
  Post process the image coming out from prediction
  '''
  if isinstance(preds, list):
    preds = preds[-1]
    if isinstance(preds, list):
      preds = preds[-1]

  # De-ensemble by averaging inferenced results.
  preds = np.array(preds[0], np.float32)

  # unpad images to get the original resolution
  new_height, new_width = preds.shape[0], preds.shape[1]
  h_start = new_height // 2 - height_even // 2
  h_end = h_start + height
  w_start = new_width // 2 - width_even // 2
  w_end = w_start + width
  preds = preds[h_start:h_end, w_start:w_end, :]
  return np.array((np.clip(preds, 0., 1.) * 255.).astype(jnp.uint8))

In [6]:
import numpy as np
from PIL import Image
import importlib
import ml_collections
import tempfile
import jax.numpy as jnp
import flax
from cog import BasePredictor, Path, Input, BaseModel

from maxim.run_eval import (
    _MODEL_FILENAME,
    _MODEL_VARIANT_DICT,
    _MODEL_CONFIGS,
    get_params,
    mod_padding_symmetric,
    make_shape_even,
    augment_image,
)


class Predictor(BasePredictor):
    def setup(self):

        self.params = {
            "Image Denoising": get_params("checkpoints/denoising-SIDD/checkpoint.npz"),
            "Image Deblurring (GoPro)": get_params(
                "checkpoints/debluring-GoPro/checkpoint.npz"
            ),
            "Image Deblurring (REDS)": get_params(
                "checkpoints/debluring-REDS/checkpoint.npz"
            ),
            "Image Deblurring (RealBlur_R)": get_params(
                "checkpoints/debluring-Real-Blur-R/checkpoint.npz"
            ),
            "Image Deblurring (RealBlur_J)": get_params(
                "checkpoints/debluring-Real-Blur-J/checkpoint.npz"
            ),
            "Image Deraining (Rain streak)": get_params(
                "checkpoints/deraining-Rain13k/checkpoint.npz"
            ),
            "Image Deraining (Rain drop)": get_params(
                "checkpoints/deraining-Raindrop/checkpoint.npz"
            ),
            "Image Dehazing (Indoor)": get_params(
                "checkpoints/dehazing-RESIDE-Indoor/checkpoint.npz"
            ),
            "Image Dehazing (Outdoor)": get_params(
                "checkpoints/dehazing-RESIDE-Outdoor/checkpoint.npz"
            ),
            "Image Enhancement (Low-light)": get_params(
                "checkpoints/enhancement-LOL/checkpoint.npz"
            ),
            "Image Enhancement (Retouching)": get_params(
                "checkpoints/enhancement-FiveK/checkpoint.npz"
            ),
        }

        model_mod = importlib.import_module(f"maxim.models.{_MODEL_FILENAME}")
        self.models = {}
        for task in _MODEL_VARIANT_DICT.keys():
            model_configs = ml_collections.ConfigDict(_MODEL_CONFIGS)
            model_configs.variant = _MODEL_VARIANT_DICT[task]
            self.models[task] = model_mod.Model(**model_configs)

    def predict(
        self,
        model: str = Input(
            choices=[
                "Image Denoising",
                "Image Deblurring (GoPro)",
                "Image Deblurring (REDS)",
                "Image Deblurring (RealBlur_R)",
                "Image Deblurring (RealBlur_J)",
                "Image Deraining (Rain streak)",
                "Image Deraining (Rain drop)",
                "Image Dehazing (Indoor)",
                "Image Dehazing (Outdoor)",
                "Image Enhancement (Low-light)",
                "Image Enhancement (Retouching)",
            ],
            description="Choose a model.",
        ),
        image: Path = Input(
            description="Input image.",
        ),
    ) -> Path:

        params = self.params[model]
        task = model.split()[1]
        model = self.models[task]

        input_img = (
            np.asarray(Image.open(str(image)).convert("RGB"), np.float32) / 255.0
        )

        # Padding images to have even shapes
        height, width = input_img.shape[0], input_img.shape[1]
        input_img = make_shape_even(input_img)
        height_even, width_even = input_img.shape[0], input_img.shape[1]

        # padding images to be multiplies of 64
        input_img = mod_padding_symmetric(input_img, factor=64)
        input_img = np.expand_dims(input_img, axis=0)

        # handle multi-stage outputs, obtain the last scale output of last stage
        preds = model.apply({"params": flax.core.freeze(params)}, input_img)
        if isinstance(preds, list):
            preds = preds[-1]
            if isinstance(preds, list):
                preds = preds[-1]

        preds = np.array(preds[0], np.float32)

        # unpad images to get the original resolution
        new_height, new_width = preds.shape[0], preds.shape[1]
        h_start = new_height // 2 - height_even // 2
        h_end = h_start + height
        w_start = new_width // 2 - width_even // 2
        w_end = w_start + width
        preds = preds[h_start:h_end, w_start:w_end, :]

        # save files
        out_path = Path(tempfile.mkdtemp()) / "output.png"
        Image.fromarray(
            np.array((np.clip(preds, 0.0, 1.0) * 255.0).astype(jnp.uint8))
        ).save(str(out_path))

        return out_path

In [7]:
from cog import Path
from pathlib import Path as PPath
import time
start = time.time()

# Assuming Predictor class definition is here

predictor = Predictor()
predictor.setup()
end = time.time()
print(f'time cost: {end-start}s')

time cost: 37.54647970199585s


In [8]:
import yaml
from flask import Flask, escape, request, send_file, jsonify
from omegaconf import OmegaConf
from werkzeug.utils import secure_filename
from PIL import Image
from cog import Path
from pathlib import Path as PPath
"""
Remove any leading/trailing whitespace.
Remove any invalid characters, such as slashes or non-ASCII characters.
Ensure the filename does not start with a period or a tilde, which are typically hidden files on Unix-based systems.
If the cleaned filename is empty or only contains a dot, it will be replaced with a random string to prevent empty or hidden filenames.
"""
import os
flask_app = Flask(__name__)
# request.files['image'] 파일 담긴 곳
# request.data 그냥 데이터 담긴 곳 => b''
# request.form 폼 데이터 담긴 곳 => ImmutableMultiDict([('type', '1')])

/tmp/ipykernel_1279458/1539194618.py:2: DeprecationWarning: 'flask.escape' is deprecated and will be removed in Flask 2.4. Import 'markupsafe.escape' instead.
  from flask import Flask, escape, request, send_file, jsonify


In [9]:
@flask_app.route('/flask1/maxim/<type>', methods=['POST'])
def maxim_response(type):
    maxim_models = [    "Image Denoising",               # 0
                        "Image Deblurring (GoPro)",      # 1
                        "Image Deblurring (REDS)",       # 2
                        "Image Deblurring (RealBlur_R)", # 3
                        "Image Deblurring (RealBlur_J)", # 4
                        "Image Deraining (Rain streak)", # 5
                        "Image Deraining (Rain drop)",   # 6
                        "Image Dehazing (Indoor)",       # 7
                        "Image Dehazing (Outdoor)",      # 8
                        "Image Enhancement (Low-light)", # 9
                        "Image Enhancement (Retouching)"]# 10
    if request.method == 'POST':
        # Check if the request contains an image
        if 'image_1' not in request.files:
            return "No image file found in the request", 400

        # Save the uploaded image
        image = request.files['image_1']
        filename = secure_filename(image.filename)
        image_path = os.path.join('uploads', filename)
        image.save(image_path)
        model_type = int(type)
        
        output_image_path = predictor.predict(model=maxim_models[model_type], image=Path(image_path))
        # Return the saved image as a response
        if (image_path.split(".")[-1] == "png"):
            return send_file(PPath(output_image_path), mimetype='image/png')
        return send_file(PPath(output_image_path), mimetype='image/jpeg')

In [ ]:
if __name__ == "__main__":
    flask_app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://70.12.130.121:5000
Press CTRL+C to quit


Image dimensions: (246, 284, 3)
Padding values: 64
sdadasdsadsdsaasdsas


13.124.65.156 - - [19/May/2023 00:33:03] "POST /flask1/maxim/5 HTTP/1.1" 200 -


Image dimensions: (250, 290, 3)
Padding values: 64
sdadasdsadsdsaasdsas


13.124.65.156 - - [19/May/2023 00:33:44] "POST /flask1/maxim/1 HTTP/1.1" 200 -


Image dimensions: (1024, 1024, 3)
Padding values: 64
sdadasdsadsdsaasdsas


13.124.65.156 - - [19/May/2023 01:18:07] "POST /flask1/maxim/3 HTTP/1.1" 200 -


In [ ]:
#찌꺼기 코드

In [ ]:
# #gpu 사용 확인
# from tensorflow.python.client import device_lib
# import torch, jax
# import tensorflow as tf
# print(tf.__version__)
# print(tf.config.list_physical_devices('GPU'))
# print(device_lib.list_local_devices())
# print(torch.cuda.is_available())
# print(jax.devices())
# # pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# print("------------")
# # GPU 할당 변경하기
# GPU_NUM = 0 # 원하는 GPU 번호 입력
# device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(device) # change allocation of current GPU

# print ('# Current cuda device: ', torch.cuda.current_device()) # check
# if torch.cuda.is_available():
#     torch.set_default_tensor_type(torch.cuda.FloatTensor)
#     print(f"using cuda: {GPU_NUM}, {torch.cuda.get_device_name(GPU_NUM)}")
# x = torch.Tensor([1.0])
# print(x.type())

In [ ]:
# start = time.time()
# image_path = "./shake.png"
# model = "Image Deblurring (RealBlur_R)"
# #                 "Image Denoising"
# #                 "Image Deblurring (GoPro)"
# #                 "Image Deblurring (REDS)"
# #                 "Image Deblurring (RealBlur_R)"
# #                 "Image Deblurring (RealBlur_J)"
# #                 "Image Deraining (Rain streak)"
# #                 "Image Deraining (Rain drop)"
# #                 "Image Dehazing (Indoor)"
# #                 "Image Dehazing (Outdoor)"
# #                 "Image Enhancement (Low-light)"
# #                 "Image Enhancement (Retouching)"
# output_image_path = predictor.predict(model=model, image=Path(image_path))

# # Load the enhanced image using PIL and display or save it
# from PIL import Image

# enhanced_image = Image.open(PPath(output_image_path))
# enhanced_image.show()
# enhanced_image.save("enhanced_test_scs.png")
# end = time.time()
# print(f'time cost: {end-start}s')

In [ ]:
# weight_drive_path = 'https://drive.google.com/uc?id=1-BRKozXh81PtwoMZ9QN3kCAieLzozHIq' # Path of the weights file which in the Google Drive
# MODEL_PATH = './adobe.npz' # name of the model to be saved as

# gdown.download(weight_drive_path, MODEL_PATH, quiet=False) # Download Model weights to your current instance


# FLAGS = DummyFlags(ckpt_path = MODEL_PATH, task = "Deblurring") # Path to your checkpoint and task name
# #     'Denoising': 'S-3',
# #     'Deblurring': 'S-3',
# #     'Deraining': 'S-2',
# #     'Dehazing': 'S-2',
# #     'Enhancement': 'S-2',
# params = get_params(FLAGS.ckpt_path) # Parse the config

# model = build_model() # Build Model

# import time
# start = time.time()
# image_path = "./123.png" # your image path
# input_img, height, width, height_even, width_even = pre_process(image_path)
# enhanced_image_array = predict(input_img) # Get predictions
# enhanced_image_array = post_process(enhanced_image_array, height, width, height_even, width_even)
# end = time.time()
# print(f'time cost: {end-start}')

# enhanced_pil_image = Image.fromarray(enhanced_image_array) # get PIL image from array
# enhanced_pil_image.save("./result/resultImage.png") # Save the image

# import matplotlib.pyplot as plt
# import matplotlib.image as img
# #강화 전
# img_test = img.imread('./shake.png')
# plt.imshow(img_test)
# plt.show()
# #강화 후
# img_test = img.imread('./result/resultImage.png')
# plt.imshow(img_test)
# plt.show()
# # 압축 models 압축 푸는 코드
# # os.system("unzip checkpoints.zip")